In [ ]:
import math
import numpy as np
import pprint as pp

from common import *
from mcs_shared import load_day, load_flight, ACCUMULATION_FLIGHTS

from bokeh.models.formatters import BasicTickFormatter

use_hvplot()

In [ ]:
%load_ext autoreload
%autoreload 2

## Snow depth map resolution

In [ ]:
RESOLUTION = 10

# Dates

In [ ]:
def diff_violin_mpl():
    data = diff.values.flatten()
    fig  = plt.figure(dpi=300, figsize=(1,2))
    ax = fig.gca()
    
    parts = ax.violinplot(
        data[~np.isnan(data)],
        quantiles=[0.05, 0.95],
        showextrema=False,
        showmeans=True
    )
    for pc in parts['bodies']:
        pc.set_facecolor('peru')
        pc.set_alpha=0.8
        pc.set_edgecolor('black')
    
    ax.set_ylim(-2, 2)
    # ax.set_yticks(np.arange(-2, 2.2, step=0.2))
    ax.yaxis.minorticks_on()
    ax.set_ylabel('Depth Difference (m)')
    ax.set_title(date)

def diff_area_mpl():
    fig  = plt.figure(dpi=300)
    ax = fig.gca()
    data.plot(vmin=-1, vmax=1, cmap='RdBu_r', cbar_kwargs={'label':'Depth Difference (m)'})
    ax.set_title('')
    ax.ticklabel_format(useOffset=False, style='plain')
    ax.tick_params(axis='x', labelrotation=45)
    ax.set_ylabel('Northing [m]')
    ax.set_xlabel('Easting [m]');

In [ ]:
def plot_flight(date):
    mcs_als, isnobal, diff, _mask = load_day(date, RESOLUTION)
    hist_bins = np.arange(0, 5, 0.1)
    
    summary_stats(mcs_als, isnobal, diff)
    
    return hv.Layout([
            # mcs_als.snowdepth.hvplot(x='x', y='y', clim=(0,3)).opts(title='ALS') + 
            # isnobal.thickness.hvplot(clim=(0,3)).opts(title='Model') + 
            diff.hvplot(x='x', y='y', cmap='bwr_r', clim=(-1,1)).opts(
                title='Depth Difference', **BOKEH_FONT, height=600, width=800,
                yformatter=BasicTickFormatter(use_scientific=False), xformatter=BasicTickFormatter(use_scientific=False),
            ),
            diff.hvplot.violin().opts(title='', inner='quartiles', 
                                      ylabel='Snow Depth (m)', xlabel='', ylim=(-2, 2),
                                      height=600, width=600, **BOKEH_FONT), 
        ] + [
            # mcs_als.snowdepth.hvplot.density(alpha=.3, label='ALS').opts(title='Depth Distribution') *
            mcs_als.snowdepth.rename('ALS').hvplot.hist(bins=hist_bins, alpha=.5, height=400, width=800, color='orange', title='Snow Depth') * \
            isnobal.thickness.rename('iSnobal').hvplot.hist(bins=hist_bins, alpha=.5, height=400, width=800, **BOKEH_FONT)
        ]).cols(2).opts(title=date, **HV_PLOT_OPTS)


def summary_stats(mcs_als, isnobal, diff):
    stats = [
        get_stats("ALS", mcs_als.snowdepth),
        get_stats("iSnobal", isnobal.thickness),
        get_stats("Difference", diff),
    ]
    display(pd.concat(stats))


def get_stats(source, data):
    return pd.DataFrame({
        "Source": [source], 
        "Median": [data.median().values.round(2)],
        "Mean": [data.mean().values.round(2)],
        "SD": [data.std().values.round(2)],
    })

# Flights

In [ ]:
plot_flight("20221208")

In [ ]:
plot_flight("20230209")

In [ ]:
plot_flight("20230316")

In [ ]:
plot_flight("20230405")

In [ ]:
plot_flight("20231228")

In [ ]:
plot_flight("20240115")